In [1]:
import pandas as pd 
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
denver_data = pd.read_csv("../datasets/denver-data.csv") 

In [2]:
# Preview the first 5 lines of the loaded data 
denver_data.head(5)

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2.016377e+09,2.016380e+15,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,2.018600e+10,2.018600e+16,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,2.016600e+10,2.016600e+16,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,2.018723e+08,2.018720e+14,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2.017411e+09,2.017410e+15,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [11]:
# Drop uneeded columns from dataset
data = denver_data.drop(['INCIDENT_ID', 'OFFENSE_ID', 'OFFENSE_CODE', 
                                    'OFFENSE_CODE_EXTENSION', 'OFFENSE_TYPE_ID',
                                     'GEO_X', 'GEO_Y', 'DISTRICT_ID',
                                    'PRECINCT_ID', 'IS_CRIME', 'IS_TRAFFIC', 'GEO_LON',
                                    'GEO_LAT', 'NEIGHBORHOOD_ID', 'INCIDENT_ADDRESS'], axis=1)
data.head(5)

,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE
0,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM
1,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM
2,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM
3,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM
4,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM


In [12]:
# Here we create a map to turn denver and vancouver crime granularity into the same level
# We basically map denvers more rich data into buckets that work for vancouver as well
crime_category_dict = {
                       'Break and Enter Commercial': 'Burglary',
                       'Break and Enter Residential/Other': 'Burglary',
                       'Homicide': 'Homicide',
                       'Mischief': 'Mischief',
                       'Offence Against a Person': 'Offence Against a Person',
                       'Other Theft': 'Theft',
                       'Theft from Vehicle': 'Theft from Vehicle',
                       'Theft of Bicycle': 'Theft', #assumptions: bicycles are not expensive enough
                       'Theft of Vehicle': 'Theft of Vehicle',
                       'Vehicle Collision or Pedestrian Struck (with Fatality)': 'Traffic Accident',
                       'Vehicle Collision or Pedestrian Struck (with Injury)': 'Traffic Accident',
                       'all-other-crimes': 'Other Crimes',
                       'larceny': 'Theft',
                       'theft-from-motor-vehicle': 'Theft from Vehicle',
                       'traffic-accident': 'Traffic Accident',
                       'drug-alcohol': 'Other Crimes', #Not sure about this one
                       'auto-theft': 'Theft of Vehicle',
                       'white-collar-crime': 'Other  Crimes', #ponzy scheme, wage fraud
                       'burglary': 'Burglary',
                       'public-disorder': 'Mischief',
                       'aggravated-assault': 'Offence Against a Person',
                       'other-crimes-against-persons': 'Offence Against a Person',
                       'robbery': 'Theft',
                       'sexual-assault': 'Offence Against a Person',
                       'murder': 'Homicide',
                       'arson': 'Mischief' #setting things on fire
                      }

In [13]:
#add crime_key
import uuid
crime_keys = []
for i in range(data.shape[0]):
    id = uuid.uuid4() 
    crime_keys.append(id)

data['crime_key'] = crime_keys
data.head(5)

,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,crime_key
0,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,eea886bf-a231-425b-ad4c-769c101d4d82
1,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,7953ed57-16be-4527-9ca2-d8438cdb0a23
2,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,a653f1ea-2c0c-4246-bc53-97db8031e9aa
3,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,22c71dbe-b6e8-4f40-8b44-21524725b093
4,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,c3b31f97-e639-4d21-b95c-1d6e4cfc60d7


In [14]:
# Map the crime type to the buckets we already came up with
new_type = []
for crime_type in data['OFFENSE_CATEGORY_ID'].values:
    new_type.append(crime_category_dict[crime_type])
data['crime_type'] = new_type
data.head(5)

,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,crime_key,crime_type
0,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,eea886bf-a231-425b-ad4c-769c101d4d82,Other Crimes
1,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,7953ed57-16be-4527-9ca2-d8438cdb0a23,Theft
2,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,a653f1ea-2c0c-4246-bc53-97db8031e9aa,Theft from Vehicle
3,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,22c71dbe-b6e8-4f40-8b44-21524725b093,Theft
4,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,c3b31f97-e639-4d21-b95c-1d6e4cfc60d7,Theft


In [15]:
# Python program to convert time 
# from 12 hour to 24 hour format
# Source: https://www.geeksforgeeks.org/python-program-convert-time-12-hour-24-hour-format/
  
# Function to convert the date format 
def convert24(str1): 
      
    # Checking if last two elements of time 
    # is AM and first two elements are 12 
    if str1[-2:] == "AM" and str1[:2] == "12": 
        return "00" + str1[2:-2] 
          
    # remove the AM     
    elif str1[-2:] == "AM": 
        return str1[:-2] 
      
    # Checking if last two elements of time 
    # is PM and first two elements are 12    
    elif str1[-2:] == "PM" and str1[:2] == "12": 
        return str1[:-2] 
          
    else: 
          
        # add 12 to hours and remove PM 
        return str(int(str1[:2]) + 12) + str1[2:8] 

In [16]:
data.head(10)

,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,crime_key,crime_type
0,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,eea886bf-a231-425b-ad4c-769c101d4d82,Other Crimes
1,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,7953ed57-16be-4527-9ca2-d8438cdb0a23,Theft
2,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,a653f1ea-2c0c-4246-bc53-97db8031e9aa,Theft from Vehicle
3,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,22c71dbe-b6e8-4f40-8b44-21524725b093,Theft
4,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,c3b31f97-e639-4d21-b95c-1d6e4cfc60d7,Theft
5,all-other-crimes,1/31/2018 12:44:00 AM,NaN,1/31/2018 1:29:00 AM,eea51f88-1ce1-467e-9177-3565f923d500,Other Crimes
6,theft-from-motor-vehicle,6/1/2017 12:15:00 PM,1/26/2018 12:15:00 PM,1/26/2018 12:24:00 PM,ed69750f-4fad-45e5-918b-9c2b1d9419d5,Theft from Vehicle
7,all-other-crimes,1/30/2018 7:40:00 AM,NaN,1/30/2018 10:33:00 AM,bb5509bf-2c99-4bdf-ae1a-6b686df5f1a4,Other Crimes
8,traffic-accident,1/30/2018 9:10:00 AM,NaN,1/30/2018 9:17:00 AM,9178f15f-fb85-42c5-ad4c-619246b6e8bb,Traffic Accident
9,theft-from-motor-vehicle,1/31/2018 12:55:00 AM,1/31/2018 6:55:00 AM,1/31/2018 7:07:00 AM,dac00ce5-71fe-425b-8c42-f346a157eb70,Theft from Vehicle


In [17]:
# Here we turn the REPORTED_DATE into a normal timestamp - "HH:MM:SS" format.
# Splitting up date into Year, Month, Day, dataset has Month/Day/Year

date_values = data['REPORTED_DATE'].values

crime_report_times = []

for date in date_values:
    if date == '-':
        crime_report_times.append(None)
    else:
        date = date.split() # Split on whitespace -> we get [mm/dd/yy, hh:mm:ss, pm/am]
        # Add a extra 0 before any hours less than 10, so from 8:15 -> 08:15
        hours = date[1].split(':')
        if len(hours[0]) == 1:
            # Create the timestamp string
            timestamp = '0'+date[1] + ' ' + date[2]    
        else:
            timestamp = date[1] + ' ' + date[2] 

        # Convert into format
        timestamp = convert24(timestamp)
        crime_report_times.append(timestamp)
    
data['crime_report_time'] = crime_report_times

In [10]:
data.head(5)

,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,crime_key,crime_type,crime_report_times
0,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,83aff1d7-7507-4685-8584-2fb5e9cc5739,Other Crimes,23:31:00
1,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,e81d48fa-8710-4987-a928-4b84d5ab68e7,Theft,17:53:00
2,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,fbd7f810-3d49-4e9c-99cf-36427be2a0d5,Theft from Vehicle,21:02:00
3,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,cf61d06f-b6aa-43cd-b66c-870d036de71c,Theft,22:29:00
4,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,d2096c40-dfbf-421d-90c8-9eb317d2e72b,Theft,16:09:00


In [18]:
# Here we turn the first_occurrence_date into a normal timestamp - "HH:MM:SS" format.
# Splitting up date into Year, Month, Day, dataset has Month/Day/Year
date_values = data['FIRST_OCCURRENCE_DATE'].values

crime_start_times = []

for date in date_values:
    date = date.split() # Split on whitespace -> we get [mm/dd/yy, hh:mm:ss, pm/am]
    
    # Add a extra 0 before any hours less than 10, so from 8:15 -> 08:15
    hours = date[1].split(':')
    if len(hours[0]) == 1:
        # Create the timestamp string
        timestamp = '0'+date[1] + ' ' + date[2]    
    else:
        timestamp = date[1] + ' ' + date[2] 
    
    # Convert into format
    timestamp = convert24(timestamp)
    crime_start_times.append(timestamp)
    
data['crime_start_time'] = crime_start_times

In [19]:
data.head(10)

,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,crime_key,crime_type,crime_report_time,crime_start_time
0,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,eea886bf-a231-425b-ad4c-769c101d4d82,Other Crimes,23:31:00,23:31:00
1,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,7953ed57-16be-4527-9ca2-d8438cdb0a23,Theft,17:53:00,12:30:00
2,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,a653f1ea-2c0c-4246-bc53-97db8031e9aa,Theft from Vehicle,21:02:00,20:00:00
3,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,22c71dbe-b6e8-4f40-8b44-21524725b093,Theft,22:29:00,19:20:00
4,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,c3b31f97-e639-4d21-b95c-1d6e4cfc60d7,Theft,16:09:00,20:53:00
5,all-other-crimes,1/31/2018 12:44:00 AM,NaN,1/31/2018 1:29:00 AM,eea51f88-1ce1-467e-9177-3565f923d500,Other Crimes,01:29:00,00:44:00
6,theft-from-motor-vehicle,6/1/2017 12:15:00 PM,1/26/2018 12:15:00 PM,1/26/2018 12:24:00 PM,ed69750f-4fad-45e5-918b-9c2b1d9419d5,Theft from Vehicle,12:24:00,12:15:00
7,all-other-crimes,1/30/2018 7:40:00 AM,NaN,1/30/2018 10:33:00 AM,bb5509bf-2c99-4bdf-ae1a-6b686df5f1a4,Other Crimes,10:33:00,07:40:00
8,traffic-accident,1/30/2018 9:10:00 AM,NaN,1/30/2018 9:17:00 AM,9178f15f-fb85-42c5-ad4c-619246b6e8bb,Traffic Accident,09:17:00,09:10:00
9,theft-from-motor-vehicle,1/31/2018 12:55:00 AM,1/31/2018 6:55:00 AM,1/31/2018 7:07:00 AM,dac00ce5-71fe-425b-8c42-f346a157eb70,Theft from Vehicle,07:07:00,00:55:00


In [21]:
# Replace all NaN values in the LAST_OCCURENCE_DATE column
data['LAST_OCCURRENCE_DATE'] = data['LAST_OCCURRENCE_DATE'].fillna('-')
data.head(5)

,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,crime_key,crime_type,crime_report_time,crime_start_time
0,all-other-crimes,6/15/2016 11:31:00 PM,-,6/15/2016 11:31:00 PM,eea886bf-a231-425b-ad4c-769c101d4d82,Other Crimes,23:31:00,23:31:00
1,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,7953ed57-16be-4527-9ca2-d8438cdb0a23,Theft,17:53:00,12:30:00
2,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,a653f1ea-2c0c-4246-bc53-97db8031e9aa,Theft from Vehicle,21:02:00,20:00:00
3,larceny,1/30/2018 7:20:00 PM,-,1/30/2018 10:29:00 PM,22c71dbe-b6e8-4f40-8b44-21524725b093,Theft,22:29:00,19:20:00
4,larceny,6/22/2017 8:53:00 PM,-,6/23/2017 4:09:00 PM,c3b31f97-e639-4d21-b95c-1d6e4cfc60d7,Theft,16:09:00,20:53:00


In [22]:
# Here we turn the last_occurrence_date into a normal timestamp - "HH:MM:SS" format.
# Splitting up date into Year, Month, Day, dataset has Month/Day/Year

date_values = data['LAST_OCCURRENCE_DATE'].values

crime_end_times = []

for date in date_values:
    if date == '-':
        crime_end_times.append(None)
    else:
        date = date.split() # Split on whitespace -> we get [mm/dd/yy, hh:mm:ss, pm/am]
        # Add a extra 0 before any hours less than 10, so from 8:15 -> 08:15
        hours = date[1].split(':')
        if len(hours[0]) == 1:
            # Create the timestamp string
            timestamp = '0'+date[1] + ' ' + date[2]    
        else:
            timestamp = date[1] + ' ' + date[2] 

        # Convert into format
        timestamp = convert24(timestamp)
        crime_end_times.append(timestamp)
    
data['crime_end_time'] = crime_end_times

In [23]:
data.head(5)

,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,crime_key,crime_type,crime_report_time,crime_start_time,crime_end_time
0,all-other-crimes,6/15/2016 11:31:00 PM,-,6/15/2016 11:31:00 PM,eea886bf-a231-425b-ad4c-769c101d4d82,Other Crimes,23:31:00,23:31:00,None
1,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,7953ed57-16be-4527-9ca2-d8438cdb0a23,Theft,17:53:00,12:30:00,16:55:00
2,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,a653f1ea-2c0c-4246-bc53-97db8031e9aa,Theft from Vehicle,21:02:00,20:00:00,08:00:00
3,larceny,1/30/2018 7:20:00 PM,-,1/30/2018 10:29:00 PM,22c71dbe-b6e8-4f40-8b44-21524725b093,Theft,22:29:00,19:20:00,None
4,larceny,6/22/2017 8:53:00 PM,-,6/23/2017 4:09:00 PM,c3b31f97-e639-4d21-b95c-1d6e4cfc60d7,Theft,16:09:00,20:53:00,None


In [ ]:
data.head(5)

In [24]:
# Drop uneeded columns from dataset
data = data.drop(['OFFENSE_CATEGORY_ID', 'FIRST_OCCURRENCE_DATE', 'LAST_OCCURRENCE_DATE', 'REPORTED_DATE'], axis=1)
data.head(5)

,crime_key,crime_type,crime_report_time,crime_start_time,crime_end_time
0,eea886bf-a231-425b-ad4c-769c101d4d82,Other Crimes,23:31:00,23:31:00,None
1,7953ed57-16be-4527-9ca2-d8438cdb0a23,Theft,17:53:00,12:30:00,16:55:00
2,a653f1ea-2c0c-4246-bc53-97db8031e9aa,Theft from Vehicle,21:02:00,20:00:00,08:00:00
3,22c71dbe-b6e8-4f40-8b44-21524725b093,Theft,22:29:00,19:20:00,None
4,c3b31f97-e639-4d21-b95c-1d6e4cfc60d7,Theft,16:09:00,20:53:00,None


In [25]:
# Finally here we convert the dataframe to a csv file to store in our repo
denver_crime_csv = data.to_csv(r'./denver-crime.csv', index = None, header=True)

In [ ]:
#End